In [1]:
import re
import keras
from keras.preprocessing.text import Tokenizer
from keras.layers import *
from keras.models import Sequential
import numpy as np

In [2]:
with open('text', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '') # убираем первый невидимый символ

In [3]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                       lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [4]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('1821', 1), ('1827', 1), ('г', 2), ('1', 8), ('озеро', 1), ('к', 64), ('меня', 26), ('на', 154), ('утре', 1), ('жизни', 9)]


In [5]:
data = tokenizer.texts_to_sequences([texts])


In [6]:
res = keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
print( res.shape )

(8940, 1000)


In [7]:
inp_words = 3
n = res.shape[0]-inp_words

In [8]:
data = tokenizer.texts_to_sequences([texts])
res = np.array( data[0] )
print( res.shape )
 
inp_words = 3
n = res.shape[0]-inp_words
 
X = np.array([res[i:i+inp_words] for i in range(n)])
Y = keras.utils.to_categorical(res[inp_words:], num_classes=maxWordsCount)


(8940,)


In [9]:
model = Sequential()
model.add(Embedding(maxWordsCount, 512, input_length = inp_words))
model.add(SimpleRNN(512, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 512)            512000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 512)               524800    
                                                                 
 dense (Dense)               (None, 1000)              513000    
                                                                 
Total params: 1,549,800
Trainable params: 1,549,800
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
280/280 [==============================] - 6s 16ms/step - loss: 6.2016 - accuracy: 0.0578
Epoch 2/50
280/280 [==============================] - 5s 16ms/step - loss: 5.4877 - accuracy: 0.0840
Epoch 3/50
280/280 [==============================] - 5s 17ms/step - loss: 4.4635 - accuracy: 0.1502
Epoch 4/50
280/280 [==============================] - 5s 16ms/step - loss: 3.3974 - accuracy: 0.2379
Epoch 5/50
280/280 [==============================] - 5s 17ms/step - loss: 2.5709 - accuracy: 0.3601
Epoch 6/50
280/280 [==============================] - 5s 17ms/step - loss: 1.9774 - accuracy: 0.4862
Epoch 7/50
280/280 [==============================] - 5s 17ms/step - loss: 1.5225 - accuracy: 0.6006
Epoch 8/50
280/280 [==============================] - 5s 17ms/step - loss: 1.1807 - accuracy: 0.6862
Epoch 9/50
280/280 [==============================] - 5s 17ms/step - loss: 0.9447 - accuracy: 0.7530
Epoch 10/50
280/280 [==============================] - 5s 17ms/step - loss: 0.7445 - accura

In [11]:
def buildPhrase(texts, str_len = 20):
  res = texts
  tokenizer.fit_on_texts([texts])
  data = tokenizer.texts_to_sequences([texts])[0]
  for i in range(str_len):
    x = data[i: i+inp_words]
    inp = np.expand_dims(x, axis=0)
 
    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    indx = pred.argmax(axis=1)[0]
    data.append(indx)
 
    res += " " + tokenizer.index_word[indx] # дописываем строку
 
  return res

In [12]:
res = buildPhrase("позитив добавляет годы")
print(res)

1/1 [==============================] - 0s 21ms/step
позитив добавляет годы шепчут ангелы ли бред как будто в огне и сказал ли бред как будто в огне и сказал ли бред


In [13]:
from random import choice
 
 
def mygen(s: str, n: int):
    for _ in range(n):
        yield choice(s.split())



m=""
for i in range(6):
    
    res = buildPhrase(next(mygen(texts, 1))+" " +next(mygen(texts, 1))+" " +next(mygen(texts, 1)))
    m+=" "+res+" \n "
print(m)

1/1 [==============================] - 0s 27ms/step
 ангел мощный былом! мной предо ли тени неведомой были да не прошли в всей видений в что я тщетно что как древней грудь 
  пережить кто и долг там где люди на не сердца ты бег со себе взгляд и не пустыне так молний светла чара не 
  рай. гирлянда и луна всегда не ах на вовек сердца в дня и в том скользит тобой ветер то моря на даже плащ 
  к Печаль грез как рой плывет лет в видишь чтоб не айрина айрина могила красота покой окно на мир ночной увы спит с 
  мной белели Вот во сне было сном все друг во мгле иль днем в дождь иль нет но идут смел как бред все 
  нахмурит мертв! -- планет прочь не его на спадали поднятые роз бросая как ответ дианы во звезд был тех что за море конца 
 
